## Install requirement libs

In [ ]:
!pip install -r ../requirements.txt

## Import libs

In [3]:
import yaml
import gdown
import pandas as pd
import numpy as np
import os.path as osp
from tqdm.notebook import tqdm

## Download the dataset

In [5]:
PROJECT_PATH = '../'
DATA_PATH = '../data'

with open(osp.join(PROJECT_PATH, 'data/links.yaml'), 'r') as stream:
    
    to_download = yaml.safe_load(stream)
    for filename, url in tqdm(to_download.items()):
        gdown.download(
            output=osp.join(PROJECT_PATH, filename),
            fuzzy=True,
            id=url,
            quiet=True,
        )

  0%|          | 0/5 [00:00<?, ?it/s]

## Read the dataset

In [6]:
train_df = pd.read_csv(
    filepath_or_buffer=osp.join(DATA_PATH, 'Train.csv'),
)

In [7]:
train_df.head()

,user_id,REGION,TENURE,MONTANT,FREQUENCE_RECH,REVENUE,ARPU_SEGMENT,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,TIGO,ZONE1,ZONE2,MRG,REGULARITY,TOP_PACK,FREQ_TOP_PACK,CHURN
0,00000bfd7d50f01092811bc0c8d7b0d6fe7c3596,FATICK,K > 24 month,4250.0,15.0,4251.0,1417.0,17.0,4.0,388.0,46.0,1.0,1.0,2.0,NO,54,On net 200F=Unlimited _call24H,8.0,0
1,00000cb4a5d760de88fecb38e2f71b7bec52e834,NaN,I 18-21 month,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,4,NaN,NaN,1
2,00001654a9d9f96303d9969d0a4a851714a4bb57,NaN,K > 24 month,3600.0,2.0,1020.0,340.0,2.0,NaN,90.0,46.0,7.0,NaN,NaN,NO,17,On-net 1000F=10MilF;10d,1.0,0
3,00001dd6fa45f7ba044bd5d84937be464ce78ac2,DAKAR,K > 24 month,13500.0,15.0,13502.0,4501.0,18.0,43804.0,41.0,102.0,2.0,NaN,NaN,NO,62,"Data:1000F=5GB,7d",11.0,0
4,000028d9e13a595abe061f9b58f3d76ab907850f,DAKAR,K > 24 month,1000.0,1.0,985.0,328.0,1.0,NaN,39.0,24.0,NaN,NaN,NaN,NO,11,Mixt 250F=Unlimited_call24H,2.0,0
